In [4]:
# Import necessary libraries
import numpy as np
import gradio as gr
from tensorflow.keras.models import load_model
import os
import warnings
warnings.filterwarnings('ignore')

# Define paths
models_path = '../models/'

# Load the trained Neural Network model
model = load_model(os.path.join(models_path, 'neural_net_tuned.h5'))

# Define meaningful column names, ranges, defaults, and stages
feature_details = {
    'Kidney_Disease_Severity': {'type': 'slider', 'range': (0, 9), 'default': 0, 'step': 1, 'label': 'Kidney Disease Severity (0 = None, 9 = Severe)'},
    'High_Blood_Pressure': {'type': 'dropdown', 'options': [0, 1], 'default': 0, 'info': '0 = No, 1 = Yes'},
    'High_Cholesterol': {'type': 'dropdown', 'options': [0, 1], 'default': 0, 'info': '0 = No, 1 = Yes'},
    'Cholesterol_Check': {'type': 'dropdown', 'options': [0, 1], 'default': 0, 'info': '0 = No, 1 = Yes'},
    'Asthma_Severity': {'type': 'slider', 'range': (0, 9), 'default': 0, 'step': 1, 'label': 'Asthma Severity (0 = None, 9 = Severe)'},
    'Chronic_Obstructive_Pulmonary_Disease_Severity': {'type': 'slider', 'range': (0, 9), 'default': 0, 'step': 1, 'label': 'Chronic Obstructive Pulmonary Disease Severity (0 = None, 9 = Severe)'},
    'Smoker': {'type': 'dropdown', 'options': [0, 1], 'default': 0, 'info': '0 = No, 1 = Yes'},
    'Stroke_History': {'type': 'dropdown', 'options': [0, 1], 'default': 0, 'info': '0 = No, 1 = Yes'},
    'Heart_Disease_or_Attack': {'type': 'dropdown', 'options': [0, 1], 'default': 0, 'info': '0 = No, 1 = Yes'},
    'Physical_Activity': {'type': 'dropdown', 'options': [0, 1], 'default': 1, 'info': '0 = No, 1 = Yes'},
    'Heavy_Alcohol_Consumption': {'type': 'dropdown', 'options': [0, 1], 'default': 0, 'info': '0 = No, 1 = Yes'},
    'Has_Healthcare': {'type': 'dropdown', 'options': [0, 1], 'default': 1, 'info': '0 = No, 1 = Yes'},
    'No_Doctor_Due_to_Cost': {'type': 'dropdown', 'options': [0, 1], 'default': 0, 'info': '0 = No, 1 = Yes'},
    'Mental_Health_Days': {'type': 'slider', 'range': (0, 30), 'default': 0, 'step': 1, 'label': 'Mental Health Days (0 = None, 30 = All)'},
    'Physical_Health_Days': {'type': 'slider', 'range': (0, 30), 'default': 0, 'step': 1, 'label': 'Physical Health Days (0 = None, 30 = All)'},
    'Difficulty_Walking': {'type': 'dropdown', 'options': [0, 1], 'default': 0, 'info': '0 = No, 1 = Yes'},
    'Sex': {'type': 'dropdown', 'options': [0, 1], 'default': 0, 'labels': ['Female', 'Male'], 'info': '0 = Female, 1 = Male'},
    'Age_Group': {'type': 'slider', 'range': (0, 13), 'default': 7, 'step': 1, 'label': 'Age Group (0 = 0, 1 = 10-14, ..., 12 = 70-79, 13 = 80+)'},
    'Education_Level': {'type': 'slider', 'range': (1, 6), 'default': 3, 'step': 1, 'label': 'Education Level (1 = Low, 6 = High)'},
    'Income_Level': {'type': 'slider', 'range': (1, 11), 'default': 6, 'step': 1, 'label': 'Income Level (1 = Low, 11 = High)'},
    'General_Health_Score': {'type': 'slider', 'range': (1, 5), 'default': 3, 'step': 1, 'label': 'General Health Score (1 = Bad, 5 = Good)'},
    'Weight_kg': {'type': 'slider', 'range': (20, 200), 'default': 70, 'step': 0.1, 'label': 'Weight (kg)'},
    'Height_cm': {'type': 'slider', 'range': (50, 250), 'default': 170, 'step': 0.1, 'label': 'Height (cm)'}
}

# Prediction label mapping
label_map = {0: 'No Diabetes', 1: 'Slightly has Diabetes', 2: 'Has Diabetes'}

# Validation function
def validate_input(**kwargs):
    for key, value in kwargs.items():
        if key in feature_details:
            details = feature_details[key]
            if 'range' in details and (value < details['range'][0] or value > details['range'][1]):
                return f"Invalid {key}: Value must be between {details['range'][0]} and {details['range'][1]}"
            elif 'options' in details and value not in details['options']:
                return f"Invalid {key}: Value must be one of {details['options']}"
    return None

# BMI calculation function
def calculate_bmi(weight_kg, height_cm):
    if height_cm <= 0:
        return 0
    bmi = weight_kg / ((height_cm / 100) ** 2)
    return round(bmi, 1)

# Prediction function
def predict_diabetes(
    Kidney_Disease_Severity, High_Blood_Pressure, High_Cholesterol, Cholesterol_Check,
    Asthma_Severity, Chronic_Obstructive_Pulmonary_Disease_Severity, Smoker, Stroke_History,
    Heart_Disease_or_Attack, Physical_Activity, Heavy_Alcohol_Consumption, Has_Healthcare,
    No_Doctor_Due_to_Cost, Mental_Health_Days, Physical_Health_Days, Difficulty_Walking,
    Sex, Age_Group, Education_Level, Income_Level, General_Health_Score, Weight_kg, Height_cm
):
    # Validate inputs
    validation_error = validate_input(
        Kidney_Disease_Severity=Kidney_Disease_Severity, High_Blood_Pressure=High_Blood_Pressure,
        High_Cholesterol=High_Cholesterol, Cholesterol_Check=Cholesterol_Check,
        Asthma_Severity=Asthma_Severity, Chronic_Obstructive_Pulmonary_Disease_Severity=Chronic_Obstructive_Pulmonary_Disease_Severity,
        Smoker=Smoker, Stroke_History=Stroke_History, Heart_Disease_or_Attack=Heart_Disease_or_Attack,
        Physical_Activity=Physical_Activity, Heavy_Alcohol_Consumption=Heavy_Alcohol_Consumption,
        Has_Healthcare=Has_Healthcare, No_Doctor_Due_to_Cost=No_Doctor_Due_to_Cost,
        Mental_Health_Days=Mental_Health_Days, Physical_Health_Days=Physical_Health_Days,
        Difficulty_Walking=Difficulty_Walking, Sex=Sex, Age_Group=Age_Group,
        Education_Level=Education_Level, Income_Level=Income_Level, General_Health_Score=General_Health_Score,
        Weight_kg=Weight_kg, Height_cm=Height_cm
    )
    if validation_error:
        return validation_error

    # Calculate BMI
    bmi = calculate_bmi(Weight_kg, Height_cm)

    # Prepare input data (placeholder for your preprocessing)
    # Assuming 48 features after one-hot encoding or similar preprocessing
    # Adjust this based on your preprocessor.joblib or training data structure
    input_data = np.array([
        Kidney_Disease_Severity, High_Blood_Pressure, High_Cholesterol, Cholesterol_Check,
        Asthma_Severity, Chronic_Obstructive_Pulmonary_Disease_Severity, bmi, Smoker, Stroke_History,
        Heart_Disease_or_Attack, Physical_Activity, Heavy_Alcohol_Consumption, Has_Healthcare,
        No_Doctor_Due_to_Cost, General_Health_Score, Mental_Health_Days, Physical_Health_Days,
        Difficulty_Walking, Sex, Age_Group, Education_Level, Income_Level,
        # Placeholder zeros for remaining features
        *[0] * (48 - 22)  # Adjust 48 based on your total feature count
    ]).reshape(1, -1)

    # Make prediction
    prediction_proba = model.predict(input_data)
    prediction = np.argmax(prediction_proba, axis=1)[0]
    confidence = prediction_proba[0, prediction]

    # Return result
    return f"Prediction: {label_map[prediction]} (Confidence: {confidence:.2f})\nCalculated BMI: {bmi}"

# Create Gradio interface with enhanced styling
with gr.Blocks(theme=gr.themes.Soft()) as interface:
    gr.Markdown("# Diabetes Prediction Tool")
    gr.Markdown("Enter your health and demographic details to predict diabetes status. Use the BMI calculator by providing weight and height. Output will be 'No Diabetes', 'Slightly has Diabetes', or 'Has Diabetes' with a confidence score.")

    with gr.Row():
        with gr.Column():
            gr.Markdown("### Health Conditions")
            Kidney_Disease_Severity = gr.Slider(minimum=0, maximum=9, step=1, value=0, label="Kidney Disease Severity (0 = None, 9 = Severe)")
            High_Blood_Pressure = gr.Dropdown(choices=[0, 1], value=0, label="High Blood Pressure", info="0 = No, 1 = Yes")
            High_Cholesterol = gr.Dropdown(choices=[0, 1], value=0, label="High Cholesterol", info="0 = No, 1 = Yes")
            Cholesterol_Check = gr.Dropdown(choices=[0, 1], value=0, label="Cholesterol Check", info="0 = No, 1 = Yes")
            Asthma_Severity = gr.Slider(minimum=0, maximum=9, step=1, value=0, label="Asthma Severity (0 = None, 9 = Severe)")
            Chronic_Obstructive_Pulmonary_Disease_Severity = gr.Slider(minimum=0, maximum=9, step=1, value=0, label="Chronic Obstructive Pulmonary Disease Severity (0 = None, 9 = Severe)")
            Smoker = gr.Dropdown(choices=[0, 1], value=0, label="Smoker", info="0 = No, 1 = Yes")
            Stroke_History = gr.Dropdown(choices=[0, 1], value=0, label="Stroke History", info="0 = No, 1 = Yes")
            Heart_Disease_or_Attack = gr.Dropdown(choices=[0, 1], value=0, label="Heart Disease or Attack", info="0 = No, 1 = Yes")
            Physical_Activity = gr.Dropdown(choices=[0, 1], value=1, label="Physical Activity", info="0 = No, 1 = Yes")
            Heavy_Alcohol_Consumption = gr.Dropdown(choices=[0, 1], value=0, label="Heavy Alcohol Consumption", info="0 = No, 1 = Yes")
            Has_Healthcare = gr.Dropdown(choices=[0, 1], value=1, label="Has Healthcare", info="0 = No, 1 = Yes")
            No_Doctor_Due_to_Cost = gr.Dropdown(choices=[0, 1], value=0, label="No Doctor Due to Cost", info="0 = No, 1 = Yes")
            Mental_Health_Days = gr.Slider(minimum=0, maximum=30, step=1, value=0, label="Mental Health Days (0 = None, 30 = All)")
            Physical_Health_Days = gr.Slider(minimum=0, maximum=30, step=1, value=0, label="Physical Health Days (0 = None, 30 = All)")
            Difficulty_Walking = gr.Dropdown(choices=[0, 1], value=0, label="Difficulty Walking", info="0 = No, 1 = Yes")

        with gr.Column():
            gr.Markdown("### Demographics and Health Scores")
            Sex = gr.Dropdown(choices=[0, 1], value=0, label="Sex", info="0 = Female, 1 = Male")
            Age_Group = gr.Slider(minimum=0, maximum=13, step=1, value=7, label="Age Group (0 = 0, 1 = 10-14, ..., 12 = 70-79, 13 = 80+)")
            Education_Level = gr.Slider(minimum=1, maximum=6, step=1, value=3, label="Education Level (1 = Low, 6 = High)")
            Income_Level = gr.Slider(minimum=1, maximum=11, step=1, value=6, label="Income Level (1 = Low, 11 = High)")
            General_Health_Score = gr.Slider(minimum=1, maximum=5, step=1, value=3, label="General Health Score (1 = Bad, 5 = Good)")
            Weight_kg = gr.Slider(minimum=20, maximum=200, step=0.1, value=70, label="Weight (kg)")
            Height_cm = gr.Slider(minimum=50, maximum=250, step=0.1, value=170, label="Height (cm)")

    # Submit button
    submit_btn = gr.Button("Predict Diabetes Status")

    # Output
    output = gr.Textbox(label="Prediction Result")

    # Connect inputs to prediction function
    submit_btn.click(
        fn=predict_diabetes,
        inputs=[
            Kidney_Disease_Severity, High_Blood_Pressure, High_Cholesterol, Cholesterol_Check,
            Asthma_Severity, Chronic_Obstructive_Pulmonary_Disease_Severity, Smoker, Stroke_History,
            Heart_Disease_or_Attack, Physical_Activity, Heavy_Alcohol_Consumption, Has_Healthcare,
            No_Doctor_Due_to_Cost, Mental_Health_Days, Physical_Health_Days, Difficulty_Walking,
            Sex, Age_Group, Education_Level, Income_Level, General_Health_Score, Weight_kg, Height_cm
        ],
        outputs=output
    )

# Launch the interface
interface.launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.
